In [ ]:
import pandas as pd
import os

# Ruta al archivo de preferencias
user_pref_path = "user_preferences.csv"
df_pref = pd.read_csv(user_pref_path)

# Diccionario: nombre de archivo -> valor de Dataset en user_preferences
files_datasets = {
    "ttcw_model_results.csv": "ttcw",
    "confederacy_model_results.csv": "confederacy",
    "pronvsprompt_model_results.csv": "pronvsprompt",
    "slm_model_results.csv": "slm",
    "hanna_model_results.csv": "hanna"
}

base_results_path = "../1_2_datasets_final/2_trained_models/by_user/rf_san_valentin"

for file_name, dataset in files_datasets.items():
    file_path = os.path.join(base_results_path, file_name)
    df = pd.read_csv(file_path)
    
    # Eliminar cualquier columna que empiece por 'ratio_ai' o 'ai_preferred'
    cols_to_drop = [col for col in df.columns if col.startswith("ratio_ai") or col.startswith("ai_preferred")]
    if cols_to_drop:
        df.drop(columns=cols_to_drop, inplace=True)
    
    # Filtrar las preferencias para el dataset indicado y seleccionar columnas necesarias
    df_pref_ds = df_pref[df_pref["Dataset"] == dataset][["User ID", "ratio_ai"]].copy()
    df_pref_ds.rename(columns={"User ID": "user_id"}, inplace=True)

    # Unir (merge) en "user_id"
    df_merged = pd.merge(df, df_pref_ds, on="user_id", how="left")
    
    # Si existe la columna "thematic_analysis.inter_theme_similarity", insertar "ratio_ai" a continuación
    if "ratio_ai" in df_merged.columns:
        if "thematic_analysis.inter_theme_similarity" in df_merged.columns:
            idx = df_merged.columns.get_loc("thematic_analysis.inter_theme_similarity")
            ratio_col = df_merged.pop("ratio_ai")
            df_merged.insert(idx + 1, "ratio_ai", ratio_col)
    
        # Crear la columna "ai_preferred": 1 si ratio_ai > 0.5, de lo contrario 0.
        ai_pref = (df_merged["ratio_ai"] > 0.5).astype(int)
        idx = df_merged.columns.get_loc("ratio_ai")
        df_merged.insert(idx + 1, "ai_preferred", ai_pref)
    else:
        print(f"Advertencia: 'ratio_ai' no existe en {file_name} después del merge.")

    output_file = file_path#.replace(".csv", "_with_ratio.csv")
    df_merged.to_csv(output_file, index=False)
    print(f"Archivo actualizado guardado en: {output_file}")

Archivo actualizado guardado en: ../1_2_datasets_final/2_trained_models/by_user/rf_san_valentin/ttcw_model_results.csv
Archivo actualizado guardado en: ../1_2_datasets_final/2_trained_models/by_user/rf_san_valentin/confederacy_model_results.csv
Archivo actualizado guardado en: ../1_2_datasets_final/2_trained_models/by_user/rf_san_valentin/pronvsprompt_model_results.csv
Archivo actualizado guardado en: ../1_2_datasets_final/2_trained_models/by_user/rf_san_valentin/slm_model_results.csv


In [3]:
import pandas as pd

# Cargar los CSV
user_preferences = pd.read_csv('/data/gmarco/canon_explicability/1_2_datasets_final/user_preferences.csv')
assessment_poems = pd.read_csv('/data/gmarco/canon_explicability/1_1_datasets_preparation/porter/output/assessment_poems.csv')

# Filtrar los ResponseId únicos y seleccionar las columnas deseadas
assessment_unique = assessment_poems[['ResponseId', 'Background in poetry', 'Frequency', 'Like poetry']].drop_duplicates(subset='ResponseId')

# Renombrar columnas para facilitar el merge:
# * 'ResponseId' → 'User ID' (clave para el merge)
# * 'Background in poetry' → 'reader_type'
assessment_unique = assessment_unique.rename(columns={
    'ResponseId': 'User ID',
    'Background in poetry': 'reader_type'
})

# Combinar usando merge left
merged = pd.merge(user_preferences, assessment_unique, on='User ID', how='left')

# Guardar el resultado en un nuevo CSV
merged.to_csv('/data/gmarco/canon_explicability/1_2_datasets_final/user_preferences_with_reader_type.csv', index=False)

In [4]:
import pandas as pd
import numpy as np

# Cargar el CSV
df = pd.read_csv('/data/gmarco/canon_explicability/1_2_datasets_final/user_preferences_with_reader_type.csv')

# Verificar si la columna 'reader_type' existe
if 'reader_type' not in df.columns:
    raise KeyError("La columna 'reader_type' no existe en el DataFrame")

# Calcular el promedio de ratio_ai para cada categoría en reader_type
reader_stats = df.groupby('reader_type')['ratio_ai'].mean().reset_index()

# Definir el umbral objetivamente, por ejemplo, la mediana de esos promedios
threshold = 0.5
print("Umbral (mediana) de ratio_ai por reader_type:", threshold)

# Mapear cada categoría: si el promedio es mayor o igual que el umbral -> 'lay reader', de lo contrario -> 'expert'
reader_stats['reader_expertise'] = np.where(reader_stats['ratio_ai'] >= threshold, 'lay reader', 'expert')

# Crear un diccionario para mapear los valores originales de reader_type
mapping = dict(zip(reader_stats['reader_type'], reader_stats['reader_expertise']))
print("Mapping de reader_type a reader_expertise:", mapping)

# Actualizar la columna reader_expertise sin sobreescribir los datos existentes:
if 'reader_expertise' in df.columns:
    mapped = df['reader_type'].map(mapping)
    df['reader_expertise'] = df['reader_expertise'].combine_first(mapped)
else:
    df['reader_expertise'] = df['reader_type'].map(mapping)

# Definir el mapeo según Dataset
dataset_mapping = {
    'hanna': 'lay reader',
    'slm': 'lay reader',
    'ttcw': 'expert',
    'pronvsprompt': 'expert',
    'confederacy': 'expert'
}

# Aplicar el mapeo y actualizar la columna reader_expertise sin sobreescribir los datos existentes:
mapped_dataset = df['Dataset'].map(dataset_mapping)
df['reader_expertise'] = df['reader_expertise'].combine_first(mapped_dataset)

# Eliminar las columnas 'reader_type', 'Frequency' y 'Like poetry'
df = df.drop(columns=['reader_type', 'Frequency', 'Like poetry'])

# Guardar el resultado
df.to_csv('/data/gmarco/canon_explicability/1_2_datasets_final/user_preferences_with_reader_expertise.csv', index=False)

Umbral (mediana) de ratio_ai por reader_type: 0.5
Mapping de reader_type a reader_expertise: {'I am a professional poet': 'expert', 'I am familiar with multiple poets and genres of poetry': 'lay reader', 'I am not very familiar with poetry': 'lay reader', 'I have read a lot of poetry, but have never written poetry': 'lay reader', 'I have written poetry, but not professionally': 'lay reader'}


In [5]:
import pandas as pd
import os

# Ruta al archivo de preferencias con reader_expertise
user_pref_path = "/data/gmarco/canon_explicability/1_2_datasets_final/user_preferences_with_reader_expertise.csv"
df_pref = pd.read_csv(user_pref_path)

for file_name, dataset in files_datasets.items():
    file_path = os.path.join(base_results_path, file_name)
    df = pd.read_csv(file_path)
    if 'reader_expertise' in df.columns:
        df = df.drop(columns=['reader_expertise'])
    # Filtrar las preferencias para el dataset indicado
    df_pref_ds = df_pref[df_pref["Dataset"] == dataset][["User ID", "reader_expertise"]].copy()
    df_pref_ds.rename(columns={"User ID": "user_id"}, inplace=True)
    
    # Unir (merge) en "user_id"
    df_merged = pd.merge(df, df_pref_ds, on="user_id", how="left")
    mapping = {'lay reader': 0, 'expert': 1}
    df_merged['expert'] = df_merged['reader_expertise'].map(mapping)
    output_file = file_path
    df_merged.to_csv(output_file, index=False)
    print(f"Archivo actualizado guardado en: {output_file}")

Archivo actualizado guardado en: ../1_2_datasets_final/2_trained_models/by_user/rf_san_valentin/ttcw_model_results.csv
Archivo actualizado guardado en: ../1_2_datasets_final/2_trained_models/by_user/rf_san_valentin/confederacy_model_results.csv
Archivo actualizado guardado en: ../1_2_datasets_final/2_trained_models/by_user/rf_san_valentin/pronvsprompt_model_results.csv
Archivo actualizado guardado en: ../1_2_datasets_final/2_trained_models/by_user/rf_san_valentin/slm_model_results.csv
